In [5]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt

We're going to compute...

### Number Theory Code

In [30]:
def gcd(a,b):
    a = abs(a)
    b = abs(b)
    if min(a,b)==0:
        return max(a,b)
    while b% a !=0:
        r = b% a
        b = a
        a = r
    return a


def gcd_list(l):
    d = 0
    for a in l:
        d = gcd(a,d)
    return d

def primefac(n):
    if n == 0:
        return {0:1}
    pf = {}
    if n < 0:
        pf[-1]=1
        n = abs(n)
    if n % 2 == 0:
        pf[2] = 0
        while n % 2 == 0:
            n = n//2
            pf[2]+=1
    if n % 3 == 0:
        pf[3] = 0
        while n % 3 == 0:
            n = n//3
            pf[3]+=1
    d = 5
    e = -1
    while d*d <= n:
        if n % d == 0:
            pf[d] = 0
            while n % d == 0:
                n = n//d
                pf[d]+=1
        d+=3+e
        e*=-1
    if n > 1:
        pf[n]=1
    return pf

def pf_to_int(pf):
    n = 1
    for p in pf:
        n*= p**pf[p]
    return n


def quad_gcd(a,b):
    pf1 = primefac(a)
    pf2 = primefac(b)
    if a == 0:
        pfgcd = {p:pf2[p]//2 for p in pf2}
        return pf_to_int(pfgcd)
    pfgcd = {}
    for p in pf1:
        if p in pf2:
            pfgcd[p] = min(pf1[p],pf2[p]//2)
    return pf_to_int(pfgcd)

def discfac(d):
    pfd = primefac(d)
    rt = pf_to_int({p:pfd[p]//2 for p in pfd})
    fd = d//(rt*rt)
    if fd % 4 > 1:
        fd*=4
        rt = rt//2
    return fd, rt

## Matrix computations
def matrixgenfromd(d):
    return np.matrix([[0,d//4],[1,-(d%4)]])

def frobmatrixfromminpoly(ap):
    a,p = ap
    d = a*a-4*p
    d0,m = discfac(d)
    one = np.matrix([[1,0],[0,1]])
    tau = matrixgenfromd(d0)
    if d0 % 4 == 0:
        return (a//2)*one+m*tau
    else:
        return ((a+m)//2) *one + m*tau

def gcdmat(mat):
    arr = np.array(mat).reshape(1,-1)[0]
    return gcd_list([int(a) for a in arr])

def matdet(m):
    m = np.array(m)
    return m[0,0]*m[1,1]-m[0,1]*m[1,0]

def mattr(m):
    m = np.array(m)
    return m[0,0]+m[1,1]

def mwexp_frommat(m):
    t = mattr(m)
    d = matdet(m)
    return abs(quad_gcd(t,d))

def matconj(m):
    return m - mattr(m)*np.matrix([[1,0],[0,1]])

def cx_to_arr(z)->np.array:
    return np.array([z.real,z.imag])

def pointgroup_roi(ap,n):
    frob = frobmatrixfromminpoly(ap)
    a,p = ap
    d = a*a-4*p
    d0,m = discfac(d)
    frobpowerminusone= np.linalg.matrix_power(frob,n)-np.matrix([[1,0],[0,1]])
    mw2exp = mwexp_frommat(frobpowerminusone)
    mwcycmat = frobpowerminusone//mw2exp
    cycorder = matdet(mwcycmat)
    if cycorder == 1:
        one_arr = np.array([1,0])
        taumat = matrixgenfromd(d0)
        tau_cp_coefs = [1,-mattr(taumat),matdet(taumat)]
        tau_cx = np.roots(tau_cp_coefs)[0]
        tau_arr = cx_to_arr(tau_cx)
        return np.array([a*one_arr+b*tau_arr
                         for a in range(mw2exp)
                         for b in range(mw2exp)])/mw2exp
    biggenorder = mw2exp*cycorder
    generator = np.array(matconj(frobpowerminusone))
    x = generator[0,0]%biggenorder
    y = generator[1,0]%biggenorder
    xygcd = gcd(x,y)
    xg,yg = (x//xygcd,y/xygcd)
    gen1intarr = np.array([xg,yg])

    #Need this to make sure we pick the second generator
    #outside of the subgroup generated by first generator
    cycgroupints = {((s*xg)%biggenorder,(s*yg)%biggenorder) 
                    for s in range(1,biggenorder)}
    if min([xy[0] for xy in cycgroupints if xy!=(0,0)]) == 0:
        gen2intarr = np.array([0,cycorder])
    else:
        gen2intarr = np.array([cycorder,0])
    xyints_all = [(a*gen1intarr+b*gen2intarr)%biggenorder 
                  for a in range(biggenorder)
                  for b in range(mw2exp)]
    one_arr = np.array([1,0])
    taumat = matrixgenfromd(d0)
    tau_cp_coefs = [1,-mattr(taumat),matdet(taumat)]
    tau_cx = np.roots(tau_cp_coefs)[0]
    tau_arr = cx_to_arr(tau_cx)
    points = np.array([xy[0]*one_arr+xy[1]*tau_arr for xy in xyints_all])/biggenorder
    return points

## Examples we want to generate

In [3]:
pointgroup_roi((-2,5),1)

array([[0.  , 0.  ],
       [0.5 , 0.  ],
       [0.25, 0.25],
       [0.75, 0.25],
       [0.5 , 0.5 ],
       [0.  , 0.5 ],
       [0.75, 0.75],
       [0.25, 0.75]])

In [7]:
ellcurvedata = pd.read_pickle('../dataframes/elliptic_curve_counts.pk')

In [8]:
ellcurvedata

,TraceFrob,Prime,Discriminant,DiscriminantFac,jInvariants,degree
ap,,,,,,
"(0, 5)",0,5,-20,"(-20, 1)",[0],2
"(1, 5)",1,5,-19,"(-19, 1)",[4],1
"(2, 5)",2,5,-16,"(-4, 2)","[1, 3]",2
"(3, 5)",3,5,-11,"(-11, 1)",[2],1
"(4, 5)",4,5,-4,"(-4, 1)",[3],1
...,...,...,...,...,...,...
"(251, 16381)",251,16381,-2523,"(-3, 29)","[6190, 6252, 6149, 8256, 6959, 1809, 10431, 10...",11
"(252, 16381)",252,16381,-2020,"(-2020, 1)","[7524, 11748, 7622, 9234, 16286, 6173, 2645, 7...",8
"(253, 16381)",253,16381,-1515,"(-1515, 1)","[7110, 8942, 10621, 1981, 9728, 3817, 4713, 58...",12


In [24]:
goodpairs = {-7:list(ellcurvedata.loc[ellcurvedata.Discriminant == -28].index)}
goodds = [-19,  -11,   -4,   -3,  -43,   -8,  -67, -163]
for d in goodds:
    goodpairs[d] = list(ellcurvedata.loc[ellcurvedata.Discriminant == d].index)
for d in goodpairs:
    goodpairs[d].sort(key= lambda x:x[1])

In [28]:
smallest_examples = [goodpairs[d][0+int(d==-7)] for d in goodpairs]

In [29]:
smallest_examples

[(4, 11), (1, 5), (3, 5), (4, 5), (5, 7), (1, 11), (6, 11), (1, 17), (1, 41)]

for d in goodds:
    os.mkdir(str(d))

In [21]:
os.mkdir(str(7))

In [31]:
smallest_examples

[(4, 11), (1, 5), (3, 5), (4, 5), (5, 7), (1, 11), (6, 11), (1, 17), (1, 41)]

In [ ]:
primeexpbounds = {p[1]:[1,p[1]] for p in smallest_examples}
for p in primeexpbounds:
    while primeexpbounds[p][1]*p<2**14:
        primeexpbounds[p][0]+=1
        primeexpbounds[p][1]*=p

In [86]:
smallest_examples_twist = [(-ap[0],ap[1]) for ap in smallest_examples]
smallest_examples_pointgroups = {ap:[pointgroup_roi(ap,n) 
                                     for n in range(1,1+primeexpbounds[ap[1]][0])] 
                                     for ap in smallest_examples+smallest_examples_twist}

In [42]:
primeexpbounds[smallest_examples[0][1]]

[4, 14641]

In [40]:
smallest_examples

[(4, 11), (1, 5), (3, 5), (4, 5), (5, 7), (1, 11), (6, 11), (1, 17), (1, 41)]

In [44]:
len(smallest_examples_pointgroups[(1,5)][-1])

15680

In [45]:
len(smallest_examples_pointgroups[(3,5)][-1])

15552

In [46]:
smallest_examples_pointgroups[(3,5)][-1][-1]

array([0.4837963 , 1.65063502])

In [47]:
str(smallest_examples_pointgroups[(3,5)][-1][-1])

'[0.4837963  1.65063502]'

In [56]:
str0 = str(smallest_examples_pointgroups[(3,5)][-1]).replace('\n',',')

In [70]:
len(smallest_examples_pointgroups[(3,5)][-1])

15552

In [71]:
len(str(len(smallest_examples_pointgroups[(3,5)][-1])))

5

In [74]:
testgrp = smallest_examples_pointgroups[(3,5)][-1]

In [73]:
len(testgrp)

15552

In [75]:
file = open('test.txt','a')
for i in range((len(testgrp)//3)+1):
    stri = ''
    for vec in testgrp[3*i:3*(i+1)]:
        stri+=str(vec)+','
    stri+='\n'
    file.write(stri)
file.close()

In [77]:
import os

In [87]:
for ap in smallest_examples_pointgroups:
    a, p = ap
    d = discfac(a*a-4*p)[0]
    imagepath = str(d)+'/'+str(a)+'_'+str(p)+'/'
    os.mkdir(imagepath)
    for n, gr in enumerate(smallest_examples_pointgroups[ap]):
        file = open(imagepath+str(n+1)+'.txt','a')
        for i in range((len(gr)//3)+1):
            stri = ''
            for vec in gr[3*i:3*(i+1)]:
                stri+=str(vec)+','
            stri+='\n'
            file.write(stri)
        file.close()


FileExistsError: [Errno 17] File exists: '-7/4_11/'